<a href="https://colab.research.google.com/github/sahil-shankar110/localRepo/blob/main/titanic_using_pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest,chi2

In [ ]:
titanic = pd.read_csv("/content/titanic.csv")

In [ ]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
titanic.drop(columns=["PassengerId","Name","Ticket","Cabin"],inplace=True)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(titanic.drop(columns=["Survived"]),titanic["Survived"],test_size=0.2,random_state=42)

In [ ]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [ ]:
y_train.sample(5)

,Survived
12,0
27,0
379,0
838,1
660,1


In [ ]:
# imputation transformer
tnf1 = ColumnTransformer([
    ("impute_age",SimpleImputer(),[2]),
    ("impute_embarked",SimpleImputer(strategy="most_frequent"),[6])
    ],remainder="passthrough")

In [ ]:
#ohe
tnf2 = ColumnTransformer([
    ("impute_sex_embarked",OneHotEncoder(sparse_output=False,handle_unknown="ignore"),[1,6])
    ],remainder="passthrough")

In [ ]:
# Scaling
tnf3 = ColumnTransformer([
    ("scale",MinMaxScaler(),slice(0,10))
    ])

In [ ]:
#feature selection
tnf4 = SelectKBest(score_func=chi2,k=8)

In [ ]:
# train model
tnf5 = DecisionTreeClassifier()

In [ ]:
pipe = Pipeline([
    ("tnf1" , tnf1),
    ("tnf2" , tnf2),
    ("tnf3" , tnf3),
    ("tnf4" , tnf4),
    ("tnf5" , tnf5)
    ])

In [ ]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('tnf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('tnf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('tnf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('tnf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7ee3571ac0e0>)),
                ('tnf5', DecisionTreeClassifier())])

In [ ]:
#
from sklearn import set_config
set_config(display="diagram")